In [1]:
# Load Drive Helper
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# Import các thư viên cần thiết
import os
import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.metrics import (precision_score, recall_score, f1_score, accuracy_score)

import tensorflow as tf

In [3]:
DATASET_PATH = '/content/drive/My Drive/NT204/dataset/'
TRAINSET_PATH = DATASET_PATH + 'train_set.csv'
TESTSET_PATH = DATASET_PATH + 'test_set.csv'

# Tạo thư mục để tải dataset
if not os.path.exists(DATASET_PATH):
  os.makedirs(DATASET_PATH)

'''
Tải tập train (train_set.csv) và tập test (test_set.csv) vào thư mục /content/drive/My Drive/NT204/dataset/ trên drive.
Lưu ý: Đặt đúng tên các file.
- Tải các tập dữ liệu tại: https://drive.google.com/drive/folders/1nYyDqbmichotvI9eSOrKo-yRwtLUy_iM?usp=sharing
'''

'\nTải tập train (train_set.csv) và tập test (test_set.csv) vào thư mục /content/drive/My Drive/NT204/dataset/ trên drive.\nLưu ý: Đặt đúng tên các file.\n- Tải các tập dữ liệu tại: https://drive.google.com/drive/folders/1nYyDqbmichotvI9eSOrKo-yRwtLUy_iM?usp=sharing\n'

In [4]:
# Tiền xử lý dữ liệu
lable_encoder = preprocessing.LabelEncoder()

# Tập train
data_train = pd.read_csv(TRAINSET_PATH)

x_train, y_train = data_train.iloc[:, :-1], data_train.iloc[:, [-1]]
y_train = y_train.apply(lable_encoder.fit_transform)
x_train, y_train = x_train.to_numpy(), y_train.to_numpy()

# Lấy số thuộc tính và số lớp
NUM_FEATURES = x_train.shape[1] # Số thuộc tính bằng số chiều (số cột của x_train hoặc x_test)
NUM_CLASSES = np.unique(y_train).size # Số class bằng tổng số các giá trị khác nhau của y_train

# one-hot encoding label
y_train = tf.keras.utils.to_categorical(y_train)

In [5]:
print("Number of features: %s" %NUM_FEATURES)
print("Number of classes: %s" %NUM_CLASSES)

Number of features: 42
Number of classes: 11


In [6]:
# Tạo mô hình deep learning
model = tf.keras.Sequential(
  [
    tf.keras.layers.Dense(32, activation="relu", input_shape=(NUM_FEATURES,)),
    tf.keras.layers.Dropout(0.15),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

# Compile mô hình
model.compile(optimizer='adam', loss = tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [7]:
# Xem thông tin của mô hình
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                1376      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 11)                363       
                                                                 
Total params: 1739 (6.79 KB)
Trainable params: 1739 (6.79 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Huấn luyện mô hình
model.fit(x=x_train, y=y_train, batch_size=1024, epochs=10, shuffle=True)

Epoch 1/10
1737/1737 [==============================] - 10s 5ms/step - loss: 0.2077 - accuracy: 0.9465
Epoch 2/10
1737/1737 [==============================] - 6s 3ms/step - loss: 0.0980 - accuracy: 0.9654
Epoch 3/10
1737/1737 [==============================] - 8s 4ms/step - loss: 0.0890 - accuracy: 0.9676
Epoch 4/10
1737/1737 [==============================] - 6s 3ms/step - loss: 0.0836 - accuracy: 0.9694
Epoch 5/10
1737/1737 [==============================] - 7s 4ms/step - loss: 0.0798 - accuracy: 0.9711
Epoch 6/10
1737/1737 [==============================] - 7s 4ms/step - loss: 0.0774 - accuracy: 0.9718
Epoch 7/10
1737/1737 [==============================] - 6s 3ms/step - loss: 0.0755 - accuracy: 0.9721
Epoch 8/10
1737/1737 [==============================] - 8s 4ms/step - loss: 0.0740 - accuracy: 0.9726
Epoch 9/10
1737/1737 [==============================] - 6s 3ms/step - loss: 0.0726 - accuracy: 0.9729
Epoch 10/10
1737/1737 [==============================] - 8s 5ms/step - loss: 0.07

In [ ]:
# Lưu mô hình
model.save('/content/drive/My Drive/NT204/model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Load dữ liệu để kiểm tra mô hình
data_test = pd.read_csv(TESTSET_PATH)
x_test, y_test = data_test.iloc[:, :-1], data_test.iloc[:, [-1]]
y_test = y_test.apply(lable_encoder.fit_transform)
x_test, y_test = x_test.to_numpy(), y_test.to_numpy()

In [ ]:
# Kiểm tra mô hình với tập test
preds = model.predict(x_test, batch_size=1024)

745/745 [==============================] - 2s 2ms/step


In [ ]:
# Tính toán kết quả
y_pred = np.argmax(preds, axis=1)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

print('Accuracy: %s\nPrecision: %s\nRecal: %s\nF1-Score: %s' %(accuracy, precision, recall, f1))

Accuracy: 0.9744243879713654
Precision: 0.969182118172714
Recal: 0.9744243879713654
F1-Score: 0.9703702365096372
